In [ ]:
pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import process,fuzz
import pandas as pd

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
import pandas as pd

# Load the review file
data = pd.read_excel('/content/LAST REVIEW.xlsx')

# Display the first few rows to understand its structure
data.head()


,Unnamed: 0,Ingredient Name EN,Repeated in recipes,Actual Match,Category,Calories,Protein,Fat,Carbohydrate,Sugar,Fiber,Unnamed: 11,ID USDA,MATCHED,usda prompt,Descrip EN
0,NaN,vegetable stock,765.0,#N/D,Processed Culinary Ingredient,17.0,0.60,0.2,2.5,NaN,NaN,"Babyfood, crackers, vegetable",NaN,NaN,Vegetable stock,"Butter, salted"
1,NaN,lasagna noodle,516.0,#¡DIV/0!,Processed food,131.0,10.10,3.1,75.0,NaN,NaN,"Pasta, corn, cooked",NaN,NaN,"Pasta, lasagna, cooked, unenriched","Butter, whipped, with salt"
2,NaN,stewing beef,416.0,"Chicken, Stewing, Giblets, Raw",Unprocessed or Minimally Processed food,250.0,26.70,18.5,0.0,NaN,NaN,"Beef, flank, steak, separable lean and fat, tr...",NaN,NaN,"Beef, stewing, separable lean and fat, trimmed...","Butter oil, anhydrous"
3,NaN,pesto sauce,340.0,#N/D,Processed Culinary Ingredient,303.0,3.92,25.0,3.2,NaN,NaN,"Sauce, barbecue",NaN,NaN,Pesto sauce,"Cheese, blue"
4,NaN,grape tomato,335.0,#¡DIV/0!,Unprocessed or Minimally Processed food,18.0,1.20,0.2,3.9,NaN,NaN,"Tomatoes, green, raw",NaN,NaN,"Tomatoes, grape, raw","Cheese, brick"


In [ ]:
# Preprocess text in 'usda prompt' column
data['usda prompt preprocessed'] = data['usda prompt'].str.lower().str.replace('[^\w\s]', ' ').str.strip()

# Display the preprocessed 'usda prompt' to verify preprocessing
data[['usda prompt', 'usda prompt preprocessed']].head()


<ipython-input-19-199df2e69cf5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['usda prompt preprocessed'] = data['usda prompt'].str.lower().str.replace('[^\w\s]', ' ').str.strip()


,usda prompt,usda prompt preprocessed
0,Vegetable stock,vegetable stock
1,"Pasta, lasagna, cooked, unenriched",pasta lasagna cooked unenriched
2,"Beef, stewing, separable lean and fat, trimmed...",beef stewing separable lean and fat trimmed...
3,Pesto sauce,pesto sauce
4,"Tomatoes, grape, raw",tomatoes grape raw


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# For demonstration, assume 'Descrip EN' is the preprocessed 'Descrip EN' column
# Preprocess the 'Descrip EN' column similarly
data['Descrip EN preprocessed'] = data['Descrip EN'].str.lower().str.replace('[^\w\s]', ' ').str.strip()

# Combine all unique USDA prompts and Descrip EN into a single set for TF-IDF vectorization
all_texts = pd.concat([data['usda prompt preprocessed'], data['Descrip EN preprocessed']]).unique()

# Import the necessary module
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text
all_texts_vectors = vectorizer.fit_transform(all_texts)

# Create the text-to-vector mapping
text_to_vector = {text: vector for text, vector in zip(all_texts, all_texts_vectors.toarray())}

# Remove NaN values from all_texts
all_texts = pd.Series(all_texts)[~pd.Series(all_texts).isnull()]

# Vectorize the text
vectorizer = TfidfVectorizer()
all_texts_vectors = vectorizer.fit_transform(all_texts)

# Map each text back to its vector
text_to_vector = {text: all_texts_vectors[i] for i, text in enumerate(all_texts)}

# Find the most similar match for each 'usda prompt'
matches = []

for usda_prompt in data['usda prompt preprocessed']:
  if not pd.isna(usda_prompt):
    usda_vector = text_to_vector[usda_prompt]

    # Calculate cosine similarity with all Descrip EN entries
    similarities = cosine_similarity(usda_vector, all_texts_vectors)

    # Find the index of the best match (excluding the match with itself)
    best_match_index = np.argmax(similarities[0, len(data):]) + len(data)
    best_match_text = all_texts[best_match_index]
    best_match_score = similarities[0, best_match_index]

    matches.append((usda_prompt, best_match_text, best_match_score))

# Convert matches to a DataFrame
matches_df = pd.DataFrame(matches, columns=['usda prompt', 'Best Match Descrip EN', 'Similarity Score'])

# Display the first few rows of the matches
matches_df.head()


<ipython-input-32-543493756269>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Descrip EN preprocessed'] = data['Descrip EN'].str.lower().str.replace('[^\w\s]', ' ').str.strip()


,usda prompt,Best Match Descrip EN,Similarity Score
0,vegetable stock,macaroni vegetable cooked enriched,0.311227
1,pasta lasagna cooked unenriched,pasta mix italian lasagna unprepared,0.575441
2,beef stewing separable lean and fat trimmed...,beef loin tenderloin steak boneless separa...,0.627534
3,pesto sauce,kashi pesto pasta primavera frozen unprepared,0.459627
4,tomatoes grape raw,babyfood grape juice no sugar canned,0.347844
